In [46]:
import numpy as np
import pandas as pd
from survivors.external import ClassifWrapSA, RegrWrapSA, SAWrapSA

In [47]:
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score

# Метрики регрессии
# - RMSE ожидаемого времени
# - R^2 ожидаемого времени
# - AUC ожидаемого времени

rmse_exp_time = lambda y_tr, y_tst, pred_time, pred_sf, pred_hf, bins: np.sqrt(mean_squared_error(y_tst["time"], pred_time))
r2_exp_time = lambda y_tr, y_tst, pred_time, pred_sf, pred_hf, bins: r2_score(y_tst["time"], pred_time)
auc_exp_time = lambda y_tr, y_tst, pred_time, pred_sf, pred_hf, bins: roc_auc_score(y_tst["time"], pred_time)

In [48]:
from sklearn.metrics import roc_auc_score, log_loss, mean_squared_error
# Метрики классификации
# - AUC вероятности события
# - log-loss вероятности события
# - rmse исхода

def find_sf_at_truetime(pred_sf, event_time, bins):
    bin_idx = np.searchsorted(event_time, bins) - 1
    bin_idx = np.clip(bin_idx, 0, len(bins) - 1)
    return pred_sf[np.arange(len(event_time)), bin_idx]

# # example
# true_times = np.array([1, 19, 21, 31])
# bins = np.array([10,20,30])
# sf = np.array([[0.9, 0.8, 0.7], 
#                [0.7, 0.6, 0.5], 
#                [0.5, 0.4, 0.3],
#                [0.05, 0.04, 0.03]])
# print(find_sf_at_truetime(sf, true_times, bins))  # [0.9  0.6  0.3  0.03] 

auc_event = lambda y_tr, y_tst, pred_time, pred_sf, pred_hf, bins: roc_auc_score(y_tst["event"], find_sf_at_truetime(pred_sf, y_tst["time"], bins))
log_loss_event = lambda y_tr, y_tst, pred_time, pred_sf, pred_hf, bins: log_loss(y_tst["event"], find_sf_at_truetime(pred_sf, y_tst["time"], bins))
rmse_event = lambda y_tr, y_tst, pred_time, pred_sf, pred_hf, bins: np.sqrt(mean_squared_error(y_tst["event"], find_sf_at_truetime(pred_sf, y_tst["time"], bins)))

In [49]:
from survivors.experiments import grid as exp
import survivors.datasets as ds

l_metrics = ["CI", "IBS", "AUPRC", "RMSE_TIME", "R2_TIME", "AUC_TIME", "AUC_EVENT", "LOGLOSS_EVENT", "RMSE_EVENT"]
X, y, features, categ, _ = ds.load_pbc_dataset()
experim = exp.Experiments(folds=5, mode="CV+SAMPLE")
experim.add_new_metric("RMSE_TIME", rmse_exp_time)
experim.add_new_metric("R2_TIME", r2_exp_time)
experim.add_new_metric("AUC_TIME", auc_exp_time)
experim.add_new_metric("AUC_EVENT", auc_event)
experim.add_new_metric("LOGLOSS_EVENT", log_loss_event)
experim.add_new_metric("RMSE_EVENT", rmse_event)
experim.set_metrics(l_metrics)

In [5]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier
from lifelines import KaplanMeierFitter
from sksurv.linear_model import CoxPHSurvivalAnalysis
from survivors.tree import CRAID

experim.add_method(ClassifWrapSA(KNeighborsClassifier()), {"n_neighbors": [5]})
experim.add_method(ClassifWrapSA(RandomForestClassifier()), {"n_estimators": [10]})
experim.add_method(RegrWrapSA(KNeighborsRegressor()), {"n_neighbors": [5]})
experim.add_method(SAWrapSA(KaplanMeierFitter()), dict())
experim.add_method(SAWrapSA(CoxPHSurvivalAnalysis()), {"alpha": [100]})
experim.add_method(SAWrapSA(CRAID()), {"depth": [10]})
experim.add_method(CRAID, {"depth": [10]})
experim.run_effective(X, y, verbose=0, stratify_best=[])
df_results = experim.get_result()

<survivors.external.mlwrap.ClassifWrapSA object at 0x0000027ACF05A890> {'n_neighbors': [5]}
<survivors.external.mlwrap.ClassifWrapSA object at 0x0000027ACF05AF50> {'n_estimators': [10]}
<survivors.external.mlwrap.RegrWrapSA object at 0x0000027AD141A980> {'n_neighbors': [5]}
<survivors.external.mlwrap.SAWrapSA object at 0x0000027AD5663130> {}
<survivors.external.mlwrap.SAWrapSA object at 0x0000027AD5661A20> {'alpha': [100]}


C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\s

<survivors.external.mlwrap.SAWrapSA object at 0x0000027AD5662AA0> {'depth': [10]}
<class 'survivors.tree.decision_tree.CRAID'> {'depth': [10]}
<survivors.external.mlwrap.ClassifWrapSA object at 0x0000027AD26B26E0> {'n_neighbors': [5]}
<survivors.external.mlwrap.ClassifWrapSA object at 0x0000027AD26B3CA0> {'n_estimators': [10]}
<survivors.external.mlwrap.RegrWrapSA object at 0x0000027AD26B3C40> {'n_neighbors': [5]}
<survivors.external.mlwrap.SAWrapSA object at 0x0000027AD26B3400> {}
<survivors.external.mlwrap.SAWrapSA object at 0x0000027AD26B2530> {'alpha': [100]}


C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\s

<survivors.external.mlwrap.SAWrapSA object at 0x0000027AD26B32E0> {'depth': [10]}
<class 'survivors.tree.decision_tree.CRAID'> {'depth': [10]}
<survivors.external.mlwrap.ClassifWrapSA object at 0x0000027AD26B26E0> {'n_neighbors': [5]}
Iteration: 1/1
EXECUTION TIME OF ClassifWrapSA(KNeighborsClassifier): [0.293 0.266 0.265 0.267 0.302 0.275 0.267 0.286 0.253 0.269 0.253 0.272
 0.263 0.273 0.284 0.252 0.29  0.258 0.269 0.278], MEM [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] {'CI': [0.555, 0.513], 'IBS': [0.246, 0.241], 'AUPRC': [0.425, 0.455], 'MSE': [2425616.138, 3009287.097]}
<survivors.external.mlwrap.ClassifWrapSA object at 0x0000027AD26B3CA0> {'n_estimators': [10]}
Iteration: 1/1
EXECUTION TIME OF ClassifWrapSA(RandomForestClassifier): [0.265 0.343 0.345 0.313 0.299 0.32  0.309 0.26  0.345 0.317 0.345 0.342
 0.303 0.341 0.308 0.274 0.334 0.342 0.303 0.268], MEM [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] {'CI': [0.617, 0.614], 'IBS': [0.196, 0.186

C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\s

C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\s

C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\survive\lib\s

Iteration: 1/1
EXECUTION TIME OF SAWrapSA(CoxPHSurvivalAnalysis): [0.355 0.418 0.414 0.335 0.343 0.38  0.425 0.394 0.422 0.38  0.365 0.383
 0.38  0.377 0.349 0.413 0.382 0.366 0.382 0.335], MEM [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] {'CI': [0.666, 0.667], 'IBS': [0.131, 0.134], 'AUPRC': [0.691, 0.691], 'MSE': [1659305.905, 1766814.024]}
<survivors.external.mlwrap.SAWrapSA object at 0x0000027AD26B32E0> {'depth': [10]}
Iteration: 1/1
EXECUTION TIME OF SAWrapSA(CRAID): [1.651 1.609 1.674 1.773 1.73  1.711 1.747 1.715 1.684 1.708 1.678 1.681
 1.73  1.758 1.773 1.709 1.646 1.631 1.681 1.559], MEM [6.414 2.25  0.031 0.008 1.965 0.988 1.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    1.281 0.    0.    0.   ] {'CI': [0.625, 0.617], 'IBS': [0.214, 0.224], 'AUPRC': [0.714, 0.716], 'MSE': [2401710.228, 2699376.688]}
<class 'survivors.tree.decision_tree.CRAID'> {'depth': [10]}
Iteration: 1/1
EXECUTION TIME OF CRAID: [1.937 1.895 2.069 2.101 1.993 1.976 2.032 2.041 2.

In [6]:
df_results

,METHOD,PARAMS,TIME,CI,IBS,AUPRC,MSE,CRIT,TIMES,MEMS,MEM,CI_mean,IBS_mean,AUPRC_mean,MSE_mean
0,ClassifWrapSA(KNeighborsClassifier),{'n_neighbors': 5},1.234413,"[0.5110389610389611, 0.5311688311688312, 0.572...","[0.3177899785972074, 0.23926117737863478, 0.22...","[0.4071347142857144, 0.4892759285714288, 0.417...","[2875195.4971428574, 3070796.2092857147, 22593...",,"[0.23901057243347168, 0.24139070510864258, 0.2...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.544221,0.261694,0.428563,2.672510e+06
1,ClassifWrapSA(RandomForestClassifier),{'n_estimators': 10},1.524835,"[0.5834415584415584, 0.6704545454545454, 0.630...","[0.20812438406816028, 0.17427406454265962, 0.1...","[0.4482053214285715, 0.46070507142857137, 0.46...","[2421587.9942857143, 1739820.9342857148, 23849...",,"[0.24786972999572754, 0.3294212818145752, 0.32...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.626364,0.190776,0.455705,2.142657e+06
2,RegrWrapSA(KNeighborsRegressor),{'n_neighbors': 5},1.414418,"[0.5704545454545454, 0.5581168831168831, 0.608...","[0.42874361542697326, 0.3189030111692633, 0.30...","[0.679910643939394, 0.7332104437229439, 0.7253...","[1424609.1785714286, 1186656.4285714286, 11469...",,"[0.23729300498962402, 0.3060445785522461, 0.28...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.571558,0.358311,0.708013,1.299685e+06
3,SAWrapSA(KaplanMeierFitter),{},1.394307,"[0.5, 0.5, 0.5, 0.5, 0.5]","[0.20329872878125196, 0.18216530965070882, 0.1...","[0.6078425806496679, 0.6005546928734674, 0.607...","[1976689.964480705, 1788654.2615088334, 219485...",,"[0.24472260475158691, 0.27121639251708984, 0.3...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.500000,0.192197,0.605376,1.953460e+06
4,SAWrapSA(CoxPHSurvivalAnalysis),{'alpha': 100},1.678197,"[0.6746753246753247, 0.7032467532467532, 0.684...","[0.1401302204109691, 0.11912412566025657, 0.11...","[0.6891068122578905, 0.6969656045797799, 0.699...","[1842316.951513259, 1577546.2837628557, 176631...",,"[0.3267405033111572, 0.2817802429199219, 0.326...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.668701,0.134738,0.693439,1.781999e+06
5,SAWrapSA(CRAID),{'depth': 10},7.102680,"[0.6451298701298701, 0.662987012987013, 0.6266...","[0.2031049643034096, 0.17609808101774893, 0.22...","[0.7365212149141546, 0.7340941337882605, 0.703...","[2244500.0687337946, 2075063.6883098758, 28734...",,"[1.4453179836273193, 1.3720934391021729, 1.352...","[5.76953125, 0.00390625, 0.0, 0.0078125, 0.011...",5.792969,0.642078,0.209990,0.715095,2.433688e+06
6,CRAID,{'depth': 10},8.274698,"[0.6668831168831169, 0.688961038961039, 0.5922...","[0.21673855633594108, 0.11971061681469919, 0.1...","[0.7335604735367447, 0.797497639607317, 0.7609...","[1027434.0053542962, 745379.9369352116, 127947...",,"[1.6378607749938965, 1.746692180633545, 1.6171...","[0.02734375, 0.0, 0.0, 0.0, 0.0078125]",0.035156,0.639481,0.191457,0.758411,1.069884e+06
7,ClassifWrapSA(KNeighborsClassifier),{'n_neighbors': 5},1.386411,"[0.5110389610389611, 0.5311688311688312, 0.572...","[0.3177899785972074, 0.23926117737863478, 0.22...","[0.4071347142857144, 0.4892759285714288, 0.417...","[2875195.4971428574, 3070796.2092857147, 22593...",,"[0.23865318298339844, 0.24819111824035645, 0.2...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.544221,0.261694,0.428563,2.672510e+06
8,ClassifWrapSA(RandomForestClassifier),{'n_estimators': 10},1.494038,"[0.5613636363636364, 0.6467532467532467, 0.630...","[0.23497124837875813, 0.1591442549339761, 0.17...","[0.46963346428571423, 0.48213321428571404, 0.4...","[2936870.331428572, 1890204.7071428576, 232979...",,"[0.29004549980163574, 0.32977914810180664, 0.2...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.608247,0.196236,0.462848,2.322670e+06
9,RegrWrapSA(KNeighborsRegressor),{'n_neighbors': 5},1.264567,"[0.5704545454545454, 0.5581168831168831, 0.608...","[0.42874361542697326, 0.3189030111692633, 0.30...","[0.679910643939394, 0.7332104437229439, 0.7253...","[1424609.1785714286, 1186656.4285714286, 11469...",,"[0.2384166717529297, 0.2931070327758789, 0.243...","[0.0, 0.0, 0.0, 0.0, 

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
# from sklearn.ensemble import RandomForestClassifier
# from lifelines import KaplanMeierFitter
# from sksurv.linear_model import CoxPHSurvivalAnalysis
# from survivors.tree import CRAID

# experim.add_method(ClassifWrapSA, {"model": [KNeighborsClassifier(n_neighbors=5)]})
# experim.add_method(ClassifWrapSA, {"model": [RandomForestClassifier(n_estimators=10)]})
# experim.add_method(RegrWrapSA, {"model": [KNeighborsRegressor(n_neighbors=5)]})
# experim.add_method(SAWrapSA, {"model": [KaplanMeierFitter()]})
# experim.add_method(SAWrapSA, {"model": [CoxPHSurvivalAnalysis(alpha=100)]})
# experim.add_method(SAWrapSA, {"model": [CRAID(depth=10)]})
# # experim.add_method(RegrWrapSA(KNeighborsRegressor()), {"n_neighbors": [5]})
# # experim.add_method(SAWrapSA(KaplanMeierFitter()), dict())
# # experim.add_method(SAWrapSA(CoxPHSurvivalAnalysis()), {"alpha": [100]})
# # experim.add_method(SAWrapSA(CRAID()), {"depth": [10]})
# experim.run_effective(X, y, verbose=0, stratify_best=[])
# df_results = experim.get_result()